In [1]:
# import relevant libraries
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:,.2f}'.format)

# Importing datasets

In [2]:
# Reading data tables
cases = pd.read_csv('Cases.csv')
interactions = pd.read_csv('Interactions.csv')
installbase = pd.read_csv('Install_Base.csv')
service = pd.read_csv('Incidents.csv')

# Creating aggregated features

Before join with revenue table, we created aggregated features for Service Incidents, Cases, Interactions and Install Base table.

## Service Incidents

In [3]:
service.head()

,NO_OF_SVC_INCIDENTS,GROUP_NAME,INCIDENT_YEAR,LAST_SERVICE_EVENT_DATE,NO_OF_REPEAT_CALLS_30D,NO_OF_FTF_CALLS_30D,CUSTOMER_SITE_ID,DUNS_NUMBER,SFDC_ID
0,16,WT03,2015,9/8/2015,6,10,183312,792229767,0018000000drGQYAA2
1,15,WT05,2015,10/31/2015,8,7,519632,78711799,001C0000010IQIfIAO
2,25,NE04,2015,12/14/2015,9,16,67170,2147445,0018000000drI9QAAU
3,17,WT07,2015,10/30/2015,6,11,62480,152543450,0018000000WFcRGAA1
4,100,WT03,2015,12/29/2015,35,65,175434,55634273,0018000000drGBmAAM


In [4]:
service.shape

(42774, 9)

### Preprocessing before aggregating

In [5]:
# check duplicates
service.drop_duplicates(inplace = True)
service.shape

(42774, 9)

In [6]:
# check column names and null values
service.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42774 entries, 0 to 42773
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   NO_OF_SVC_INCIDENTS      42774 non-null  int64 
 1   GROUP_NAME               42774 non-null  object
 2   INCIDENT_YEAR            42774 non-null  int64 
 3   LAST_SERVICE_EVENT_DATE  42774 non-null  object
 4   NO_OF_REPEAT_CALLS_30D   42774 non-null  int64 
 5   NO_OF_FTF_CALLS_30D      42774 non-null  int64 
 6   CUSTOMER_SITE_ID         42774 non-null  int64 
 7   DUNS_NUMBER              42774 non-null  int64 
 8   SFDC_ID                  42774 non-null  object
dtypes: int64(6), object(3)
memory usage: 3.3+ MB


In [7]:
# drop unnecessary columns
service.drop(['DUNS_NUMBER','SFDC_ID','GROUP_NAME'],axis = 1, inplace = True)

In [8]:
service.head()

,NO_OF_SVC_INCIDENTS,INCIDENT_YEAR,LAST_SERVICE_EVENT_DATE,NO_OF_REPEAT_CALLS_30D,NO_OF_FTF_CALLS_30D,CUSTOMER_SITE_ID
0,16,2015,9/8/2015,6,10,183312
1,15,2015,10/31/2015,8,7,519632
2,25,2015,12/14/2015,9,16,67170
3,17,2015,10/30/2015,6,11,62480
4,100,2015,12/29/2015,35,65,175434


### Aggregating

In [9]:
# aggregate NO_OF_SVC_INCIDENTS, NO_OF_REPEAT_CALLS_30D, NO_OF_FTF_CALLS_30D, LAST_SERVICE_EVENT_DATE
service = service.groupby('CUSTOMER_SITE_ID').agg({'NO_OF_SVC_INCIDENTS': 'sum',
                                                   'NO_OF_REPEAT_CALLS_30D': 'sum',
                                                   'NO_OF_FTF_CALLS_30D': 'sum'}).reset_index()
service.head()

,CUSTOMER_SITE_ID,NO_OF_SVC_INCIDENTS,NO_OF_REPEAT_CALLS_30D,NO_OF_FTF_CALLS_30D
0,24,13,7,6
1,36,111,38,73
2,85,1,0,1
3,86,11,1,10
4,90,57,13,44


In [10]:
service.shape

(13581, 4)

In [11]:
# rename columns
service = service.rename(columns = {
                                    "NO_OF_SVC_INCIDENTS": "Total_SVC_Incidents",
                                    "NO_OF_REPEAT_CALLS_30D": "Total_Repeat_Calls",
                                    "NO_OF_FTF_CALLS_30D": "Total_FTF_Calls"})

In [12]:
service.head()

,CUSTOMER_SITE_ID,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls
0,24,13,7,6
1,36,111,38,73
2,85,1,0,1
3,86,11,1,10
4,90,57,13,44


In [13]:
# check all customer sites are unique
len(service['CUSTOMER_SITE_ID'])/len(service['CUSTOMER_SITE_ID'].unique())

1.0

## Interactions

In [14]:
interactions.head()

,SFDC_ID,Task_Subtype,Created_Date,Interaction_Amount,Type,Assigned,Visit,Task,Last_Modified_Date,CUSTOMER_SITE_ID
0,0018000000drGW3AAM,Call,2019-04-16 19:36:23.0000000,nan,Call,Stephen Czekanski,1,1,2019-04-16 19:47:43.0000000,"196,773.00"
1,0018000000drGW3AAM,Call,2019-06-11 20:58:44.0000000,nan,Call,April Clark,1,1,2019-06-11 20:58:44.0000000,"196,773.00"
2,0018000000drGW3AAM,Call,2019-06-12 14:35:19.0000000,nan,Call,Stephen Czekanski,1,1,2019-06-12 14:47:03.0000000,"196,773.00"
3,0018000000drGW3AAM,Call,2019-06-12 14:25:22.0000000,nan,Call,Stephen Czekanski,1,1,2019-06-12 14:31:19.0000000,"196,773.00"
4,0018000000drGW3AAM,Call,2019-07-18 15:43:21.0000000,nan,Call,Stephen Czekanski,1,1,2019-07-18 15:48:19.0000000,"196,773.00"


In [15]:
interactions.shape

(905864, 10)

### Preprocessing before aggregating

In [16]:
# check duplicates
interactions.drop_duplicates(inplace = True)
interactions.shape

(894839, 10)

In [17]:
# check column names and null values
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894839 entries, 0 to 905863
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   SFDC_ID             894839 non-null  object 
 1   Task_Subtype        894839 non-null  object 
 2   Created_Date        894839 non-null  object 
 3   Interaction_Amount  0 non-null       float64
 4   Type                792292 non-null  object 
 5   Assigned            894839 non-null  object 
 6   Visit               894839 non-null  int64  
 7   Task                894839 non-null  int64  
 8   Last_Modified_Date  894839 non-null  object 
 9   CUSTOMER_SITE_ID    652467 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 75.1+ MB


In [18]:
# drop unnecessary columns
interactions.drop('Interaction_Amount',axis = 1,inplace = True)

In [19]:
# fill in null values in Type
interactions['Type'] =  interactions['Type'].fillna('Other')

In [20]:
# drop rows with null values
interactions = interactions.dropna()

In [21]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652467 entries, 0 to 905863
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   SFDC_ID             652467 non-null  object 
 1   Task_Subtype        652467 non-null  object 
 2   Created_Date        652467 non-null  object 
 3   Type                652467 non-null  object 
 4   Assigned            652467 non-null  object 
 5   Visit               652467 non-null  int64  
 6   Task                652467 non-null  int64  
 7   Last_Modified_Date  652467 non-null  object 
 8   CUSTOMER_SITE_ID    652467 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 49.8+ MB


In [22]:
interactions.head()

,SFDC_ID,Task_Subtype,Created_Date,Type,Assigned,Visit,Task,Last_Modified_Date,CUSTOMER_SITE_ID
0,0018000000drGW3AAM,Call,2019-04-16 19:36:23.0000000,Call,Stephen Czekanski,1,1,2019-04-16 19:47:43.0000000,"196,773.00"
1,0018000000drGW3AAM,Call,2019-06-11 20:58:44.0000000,Call,April Clark,1,1,2019-06-11 20:58:44.0000000,"196,773.00"
2,0018000000drGW3AAM,Call,2019-06-12 14:35:19.0000000,Call,Stephen Czekanski,1,1,2019-06-12 14:47:03.0000000,"196,773.00"
3,0018000000drGW3AAM,Call,2019-06-12 14:25:22.0000000,Call,Stephen Czekanski,1,1,2019-06-12 14:31:19.0000000,"196,773.00"
4,0018000000drGW3AAM,Call,2019-07-18 15:43:21.0000000,Call,Stephen Czekanski,1,1,2019-07-18 15:48:19.0000000,"196,773.00"


In [23]:
# drop unnecessary columns
interactions.drop(['Task_Subtype','Last_Modified_Date','SFDC_ID','Assigned','Created_Date'],axis = 1, inplace = True)

In [24]:
interactions.head()

,Type,Visit,Task,CUSTOMER_SITE_ID
0,Call,1,1,"196,773.00"
1,Call,1,1,"196,773.00"
2,Call,1,1,"196,773.00"
3,Call,1,1,"196,773.00"
4,Call,1,1,"196,773.00"


### Aggregating

In [25]:
# aggregate Type
interactions1 = interactions[['CUSTOMER_SITE_ID','Type']]
interactions2 = interactions1.groupby('CUSTOMER_SITE_ID')['Type'].apply(lambda x: x.mode()[0]).reset_index()
interactions2.head()

,CUSTOMER_SITE_ID,Type
0,20.00,Other
1,24.00,Call
2,29.00,Dial
3,31.00,Call
4,36.00,Call


In [26]:
# # check all customer sites are unique
len(interactions2['CUSTOMER_SITE_ID'])/len(interactions2['CUSTOMER_SITE_ID'].unique())

1.0

In [27]:
# aggregate Visit and Task
interactions = interactions.drop('Type',axis = 1)
interactions3 = interactions.groupby('CUSTOMER_SITE_ID')[['Visit','Task']].sum().reset_index()
interactions3.head()

,CUSTOMER_SITE_ID,Visit,Task
0,20.00,4,7
1,24.00,81,73
2,29.00,1,1
3,31.00,25,26
4,36.00,84,88


In [28]:
# combine aggregated variables
interactions = pd.merge(interactions2, interactions3, how = 'inner')
interactions.head()

,CUSTOMER_SITE_ID,Type,Visit,Task
0,20.00,Other,4,7
1,24.00,Call,81,73
2,29.00,Dial,1,1
3,31.00,Call,25,26
4,36.00,Call,84,88


In [29]:
# rename columns
interactions = interactions.rename(columns = {"Type": 'Most_Frequent_Interaction_Type',
                                              "Visit": "Total_Visits",
                                              "Task": "Total_Tasks"})
interactions.head()

,CUSTOMER_SITE_ID,Most_Frequent_Interaction_Type,Total_Visits,Total_Tasks
0,20.00,Other,4,7
1,24.00,Call,81,73
2,29.00,Dial,1,1
3,31.00,Call,25,26
4,36.00,Call,84,88


In [30]:
# # check all customer sites are unique
len(interactions['CUSTOMER_SITE_ID'])/len(interactions['CUSTOMER_SITE_ID'].unique())

1.0

## Cases

In [31]:
cases.sort_values(by='NO_OF_CASES', ascending=False).head(10)

,NO_OF_CASES,CASE_ORIGIN,CASE_REASON,CX_CASE_L1_REASON,CX_CASE_L2_REASON,MAX_MODIFIED_DATE,DUNS_NUMBER,SFDC_ID,CUSTOMER_SITE_ID
1635,50,Email - VTI NACC,Customer Experience,NaN,NaN,4/24/2020 0:00,"51,113,330.00",0018000000drEC9AAM,20.00
2128,10,Email - VTI NACC,Customer Experience,NaN,NaN,4/27/2020 0:00,"42,971,069.00",0018000000WWwxvAAD,"64,948.00"
2828,8,Email - VTI NACC,Customer Experience,NaN,NaN,4/27/2020 0:00,"48,583,939.00",001C000001FYPD0IAP,"702,114.00"
309,8,Email - VTI NACC,Customer Experience,NaN,NaN,2/7/2020 0:00,"206,945,144.00",0018000000dsClhAAE,"112,471.00"
1258,7,Email - VTI NACC,Customer Experience,NaN,NaN,2/20/2020 0:00,"122,324,999.00",0011A00001XPTF4QAP,"8,317,382.00"
1985,7,TS Survey Followup,CX: Tech Support,NaN,NaN,2/10/2020 0:00,"51,113,330.00",0018000000drEC9AAM,20.00
980,6,Email - VTI NACC,Customer Experience,NaN,NaN,2/27/2020 0:00,"115,993,883.00",001C000001J9SyxIAF,"736,520.00"
1671,6,Email - VTI NACC,Customer Experience,NaN,NaN,3/12/2020 0:00,"15,772,159.00",0018000000drJccAAE,"206,712.00"
153,6,Email - VTI NACC,Customer Experience,NaN,NaN,12/20/2019 0:00,"246,997,816.00",0018000000drEkeAAE,"72,087.00"
2011,6,Email - VTI NACC,Customer Experience,NaN,NaN,4/23/2020 0:00,"80,742,445.00",0011A00001VbI2SQAV,"7,096,062.00"


In [32]:
cases.shape

(3027, 9)

### Preprocessing before aggregating

In [33]:
# check duplicates
cases.drop_duplicates(inplace = True)
cases.shape

(3018, 9)

In [34]:
# check column names and null values
cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3018 entries, 0 to 3026
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NO_OF_CASES        3018 non-null   int64  
 1   CASE_ORIGIN        3016 non-null   object 
 2   CASE_REASON        3018 non-null   object 
 3   CX_CASE_L1_REASON  571 non-null    object 
 4   CX_CASE_L2_REASON  426 non-null    object 
 5   MAX_MODIFIED_DATE  3018 non-null   object 
 6   DUNS_NUMBER        2950 non-null   float64
 7   SFDC_ID            2948 non-null   object 
 8   CUSTOMER_SITE_ID   2950 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 235.8+ KB


In [35]:
# drop rows with null values in CUSTOMER_SITE_ID, CASE_ORIGIN
cases = cases.dropna(axis = 0, subset = ['CUSTOMER_SITE_ID'])
cases = cases.dropna(axis = 0, subset = ['CASE_ORIGIN'])
cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 0 to 3026
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NO_OF_CASES        2948 non-null   int64  
 1   CASE_ORIGIN        2948 non-null   object 
 2   CASE_REASON        2948 non-null   object 
 3   CX_CASE_L1_REASON  555 non-null    object 
 4   CX_CASE_L2_REASON  416 non-null    object 
 5   MAX_MODIFIED_DATE  2948 non-null   object 
 6   DUNS_NUMBER        2948 non-null   float64
 7   SFDC_ID            2946 non-null   object 
 8   CUSTOMER_SITE_ID   2948 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 230.3+ KB


In [36]:
# drop unnecessary columns
cases.drop(['SFDC_ID','DUNS_NUMBER','MAX_MODIFIED_DATE','CX_CASE_L1_REASON','CX_CASE_L2_REASON'],
            axis = 1, inplace = True)
cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 0 to 3026
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NO_OF_CASES       2948 non-null   int64  
 1   CASE_ORIGIN       2948 non-null   object 
 2   CASE_REASON       2948 non-null   object 
 3   CUSTOMER_SITE_ID  2948 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 115.2+ KB


In [37]:
cases.head()

,NO_OF_CASES,CASE_ORIGIN,CASE_REASON,CUSTOMER_SITE_ID
0,1,CC Survey Followup,CX: Customer Care,"10,599.00"
1,1,CC Survey Followup,CX: Customer Care,"73,688.00"
2,1,CC Survey Followup,CX: Customer Care,895.00
3,1,CC Survey Followup,CX: Customer Care,"229,512.00"
4,1,TS Survey Followup,CX: Customer Care,"362,868.00"


### Aggregating

In [38]:
# aggregate CASE_ORIGIN
cases01 = cases.groupby(['CUSTOMER_SITE_ID','CASE_ORIGIN'])['NO_OF_CASES'].sum().reset_index()
cases02 = cases01.groupby('CUSTOMER_SITE_ID')['NO_OF_CASES'].max().reset_index()
cases03 = cases02.merge(cases01, how = 'inner', on=['CUSTOMER_SITE_ID','NO_OF_CASES'])
cases03 = cases03.groupby('CUSTOMER_SITE_ID')['CASE_ORIGIN'].apply(lambda x: x.mode()[0]).reset_index()
cases03.head(10)

,CUSTOMER_SITE_ID,CASE_ORIGIN
0,20.00,Email - VTI NACC
1,36.00,Email - VTI NACC
2,86.00,Email - VTI NACC
3,114.00,Email - VTI NACC
4,126.00,Phone
5,136.00,Phone
6,169.00,Phone
7,367.00,TS Survey Followup
8,403.00,Phone
9,429.00,Email - VTI NACC


In [39]:
len(cases03.CUSTOMER_SITE_ID.value_counts())/len(cases03)

1.0

In [40]:
# aggregate CASE_REASON
cases11 = cases.groupby(['CUSTOMER_SITE_ID','CASE_REASON'])['NO_OF_CASES'].sum().reset_index()
cases12 = cases11.groupby('CUSTOMER_SITE_ID')['NO_OF_CASES'].max().reset_index()
cases13 = cases12.merge(cases11, how = 'inner', on=['CUSTOMER_SITE_ID','NO_OF_CASES'])
cases13 = cases13.groupby('CUSTOMER_SITE_ID')['CASE_REASON'].apply(lambda x: x.mode()[0]).reset_index()
cases13.head(10)

,CUSTOMER_SITE_ID,CASE_REASON
0,20.00,Customer Experience
1,36.00,Customer Experience
2,86.00,Customer Experience
3,114.00,Customer Experience
4,126.00,Customer Experience
5,136.00,Customer Experience
6,169.00,Customer Experience
7,367.00,CX: Tech Support
8,403.00,Customer Experience
9,429.00,Customer Experience


In [41]:
len(cases13.CUSTOMER_SITE_ID.value_counts())/len(cases13)

1.0

In [42]:
# aggregate NO_OF_CASES
cases1 = cases[['CUSTOMER_SITE_ID','NO_OF_CASES']]
cases2 = cases1.groupby('CUSTOMER_SITE_ID')['NO_OF_CASES'].sum().reset_index()
cases = cases2
cases2.head()

,CUSTOMER_SITE_ID,NO_OF_CASES
0,20.00,83
1,36.00,2
2,86.00,1
3,114.00,1
4,126.00,1


In [43]:
# rename columns
cases = cases.rename(columns = {"NO_OF_CASES": "Total_Cases"})
cases.head()

,CUSTOMER_SITE_ID,Total_Cases
0,20.00,83
1,36.00,2
2,86.00,1
3,114.00,1
4,126.00,1


In [44]:
cases = pd.merge(cases, cases03, on='CUSTOMER_SITE_ID').merge(cases13, on='CUSTOMER_SITE_ID')

In [45]:
cases = cases.rename(columns = {'CASE_ORIGIN':'Max_Case_Origin','CASE_REASON': 'Max_Case_Reason'})

In [46]:
cases.head()

,CUSTOMER_SITE_ID,Total_Cases,Max_Case_Origin,Max_Case_Reason
0,20.00,83,Email - VTI NACC,Customer Experience
1,36.00,2,Email - VTI NACC,Customer Experience
2,86.00,1,Email - VTI NACC,Customer Experience
3,114.00,1,Email - VTI NACC,Customer Experience
4,126.00,1,Phone,Customer Experience


In [47]:
# # check all customer sites are unique
len(cases['CUSTOMER_SITE_ID'])/len(cases['CUSTOMER_SITE_ID'].unique())

1.0

## Install Base

In [48]:
installbase.head()

,INSTANCE_ID,STATUS,INSTALL_DATE,CONTRACT_FLAG,CONTRACT_START_DATE,CONTRACT_END_DATE,DUNS_NUMBER,CUSTOMER_SITE_ID,SITE_NUMBER,SFDC_ID,PRODUCT_FAMILY,INSTANCE_LAST_UPDATE_DATE,CONTRACT_CATEGORY
0,225594,Inactive - Unconfirmed,5/7/2004,N,NaN,NaN,137361531,10970,13140518,0018000000drGe2AAE,GRAPHICS,2/17/2004,NaN
1,214815,Inactive - Unconfirmed,4/16/2004,N,NaN,NaN,49061187,10983,100026717,0018000000drEebAAE,GRAPHICS,3/15/2004,NaN
2,159150,Inactive - Unconfirmed,4/23/2004,N,NaN,NaN,137361531,10970,13140518,0018000000drGe2AAE,GRAPHICS,4/23/2004,NaN
3,199246,Inactive - Unconfirmed,8/16/2004,N,NaN,NaN,85497652,10967,13140513,0018000000drGe0AAE,GRAPHICS,3/24/2004,NaN
4,199247,Inactive - Unconfirmed,7/23/2004,N,NaN,NaN,85497652,10967,13140513,0018000000drGe0AAE,GRAPHICS,3/24/2004,NaN


In [49]:
installbase.shape

(120450, 13)

### Preprocessing before aggregating

In [50]:
# check duplicates
installbase.drop_duplicates(inplace = True)
installbase.shape

(120149, 13)

In [51]:
# check column names and null values
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120149 entries, 0 to 120449
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   INSTANCE_ID                120149 non-null  int64 
 1   STATUS                     119629 non-null  object
 2   INSTALL_DATE               120032 non-null  object
 3   CONTRACT_FLAG              120149 non-null  object
 4   CONTRACT_START_DATE        38425 non-null   object
 5   CONTRACT_END_DATE          38425 non-null   object
 6   DUNS_NUMBER                120149 non-null  int64 
 7   CUSTOMER_SITE_ID           120149 non-null  int64 
 8   SITE_NUMBER                120149 non-null  object
 9   SFDC_ID                    120148 non-null  object
 10  PRODUCT_FAMILY             120149 non-null  object
 11  INSTANCE_LAST_UPDATE_DATE  120149 non-null  object
 12  CONTRACT_CATEGORY          38425 non-null   object
dtypes: int64(3), object(10)
memory usage: 12.8+ 

In [52]:
# copy INSTALL_DATE for later steps
installbase['INSTALL_DATE1'] = installbase['INSTALL_DATE']

In [53]:
# convert date variables to datetime data type
installbase[["INSTALL_DATE","INSTALL_DATE1","CONTRACT_START_DATE", "CONTRACT_END_DATE","INSTANCE_LAST_UPDATE_DATE"]] = installbase[["INSTALL_DATE","INSTALL_DATE1","CONTRACT_START_DATE", "CONTRACT_END_DATE","INSTANCE_LAST_UPDATE_DATE"]].apply(pd.to_datetime)

In [54]:
# check data types
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120149 entries, 0 to 120449
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   INSTANCE_ID                120149 non-null  int64         
 1   STATUS                     119629 non-null  object        
 2   INSTALL_DATE               120032 non-null  datetime64[ns]
 3   CONTRACT_FLAG              120149 non-null  object        
 4   CONTRACT_START_DATE        38425 non-null   datetime64[ns]
 5   CONTRACT_END_DATE          38425 non-null   datetime64[ns]
 6   DUNS_NUMBER                120149 non-null  int64         
 7   CUSTOMER_SITE_ID           120149 non-null  int64         
 8   SITE_NUMBER                120149 non-null  object        
 9   SFDC_ID                    120148 non-null  object        
 10  PRODUCT_FAMILY             120149 non-null  object        
 11  INSTANCE_LAST_UPDATE_DATE  120149 non-null  datetime

In [55]:
# create a new column called Contract_length
installbase['Contract_length'] = (installbase['CONTRACT_END_DATE'] - installbase['CONTRACT_START_DATE']).dt.days

# fill missing value in this column using 0
installbase['Contract_length'] = installbase['Contract_length'].fillna(0)

In [56]:
# drop unnecessary columns
installbase.drop(['INSTANCE_ID','SFDC_ID','SITE_NUMBER','DUNS_NUMBER','PRODUCT_FAMILY','CONTRACT_START_DATE','CONTRACT_END_DATE'], 
                 axis = 1, inplace = True)
installbase.head()

,STATUS,INSTALL_DATE,CONTRACT_FLAG,CUSTOMER_SITE_ID,INSTANCE_LAST_UPDATE_DATE,CONTRACT_CATEGORY,INSTALL_DATE1,Contract_length
0,Inactive - Unconfirmed,2004-05-07,N,10970,2004-02-17,NaN,2004-05-07,0.00
1,Inactive - Unconfirmed,2004-04-16,N,10983,2004-03-15,NaN,2004-04-16,0.00
2,Inactive - Unconfirmed,2004-04-23,N,10970,2004-04-23,NaN,2004-04-23,0.00
3,Inactive - Unconfirmed,2004-08-16,N,10967,2004-03-24,NaN,2004-08-16,0.00
4,Inactive - Unconfirmed,2004-07-23,N,10967,2004-03-24,NaN,2004-07-23,0.00


In [57]:
# check column names and null values
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120149 entries, 0 to 120449
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   STATUS                     119629 non-null  object        
 1   INSTALL_DATE               120032 non-null  datetime64[ns]
 2   CONTRACT_FLAG              120149 non-null  object        
 3   CUSTOMER_SITE_ID           120149 non-null  int64         
 4   INSTANCE_LAST_UPDATE_DATE  120149 non-null  datetime64[ns]
 5   CONTRACT_CATEGORY          38425 non-null   object        
 6   INSTALL_DATE1              120032 non-null  datetime64[ns]
 7   Contract_length            120149 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int64(1), object(3)
memory usage: 8.2+ MB


In [58]:
installbase['CONTRACT_FLAG'].value_counts()

N    81724
Y    38425
Name: CONTRACT_FLAG, dtype: int64

In [59]:
# substitute Y/N with 1/0 in CONTRACT_FLAG
installbase['CONTRACT_FLAG'] = installbase['CONTRACT_FLAG'].apply(lambda x: 1 if x=='Y' else 0)

In [60]:
installbase.head()

,STATUS,INSTALL_DATE,CONTRACT_FLAG,CUSTOMER_SITE_ID,INSTANCE_LAST_UPDATE_DATE,CONTRACT_CATEGORY,INSTALL_DATE1,Contract_length
0,Inactive - Unconfirmed,2004-05-07,0,10970,2004-02-17,NaN,2004-05-07,0.00
1,Inactive - Unconfirmed,2004-04-16,0,10983,2004-03-15,NaN,2004-04-16,0.00
2,Inactive - Unconfirmed,2004-04-23,0,10970,2004-04-23,NaN,2004-04-23,0.00
3,Inactive - Unconfirmed,2004-08-16,0,10967,2004-03-24,NaN,2004-08-16,0.00
4,Inactive - Unconfirmed,2004-07-23,0,10967,2004-03-24,NaN,2004-07-23,0.00


In [61]:
# check column names and null values
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120149 entries, 0 to 120449
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   STATUS                     119629 non-null  object        
 1   INSTALL_DATE               120032 non-null  datetime64[ns]
 2   CONTRACT_FLAG              120149 non-null  int64         
 3   CUSTOMER_SITE_ID           120149 non-null  int64         
 4   INSTANCE_LAST_UPDATE_DATE  120149 non-null  datetime64[ns]
 5   CONTRACT_CATEGORY          38425 non-null   object        
 6   INSTALL_DATE1              120032 non-null  datetime64[ns]
 7   Contract_length            120149 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int64(2), object(2)
memory usage: 8.2+ MB


In [62]:
# drop rows with null values in STATUS, INSTALL_DATE
installbase = installbase.dropna(axis = 0, subset = ['STATUS'])
installbase = installbase.dropna(axis = 0, subset = ['INSTALL_DATE'])

In [63]:
# check column names and null values
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119512 entries, 0 to 120449
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   STATUS                     119512 non-null  object        
 1   INSTALL_DATE               119512 non-null  datetime64[ns]
 2   CONTRACT_FLAG              119512 non-null  int64         
 3   CUSTOMER_SITE_ID           119512 non-null  int64         
 4   INSTANCE_LAST_UPDATE_DATE  119512 non-null  datetime64[ns]
 5   CONTRACT_CATEGORY          38348 non-null   object        
 6   INSTALL_DATE1              119512 non-null  datetime64[ns]
 7   Contract_length            119512 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int64(2), object(2)
memory usage: 8.2+ MB


In [64]:
installbase.CONTRACT_CATEGORY.value_counts()

FSMA          33496
Full Care      4789
Supportive       55
WFC               8
Name: CONTRACT_CATEGORY, dtype: int64

In [65]:
# fill in null values in CONTRACT_CATEGORY
installbase['CONTRACT_CATEGORY'] =  installbase['CONTRACT_CATEGORY'].fillna('No Contract')

In [66]:
# check column names and null values
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119512 entries, 0 to 120449
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   STATUS                     119512 non-null  object        
 1   INSTALL_DATE               119512 non-null  datetime64[ns]
 2   CONTRACT_FLAG              119512 non-null  int64         
 3   CUSTOMER_SITE_ID           119512 non-null  int64         
 4   INSTANCE_LAST_UPDATE_DATE  119512 non-null  datetime64[ns]
 5   CONTRACT_CATEGORY          119512 non-null  object        
 6   INSTALL_DATE1              119512 non-null  datetime64[ns]
 7   Contract_length            119512 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int64(2), object(2)
memory usage: 8.2+ MB


In [67]:
installbase.head()

,STATUS,INSTALL_DATE,CONTRACT_FLAG,CUSTOMER_SITE_ID,INSTANCE_LAST_UPDATE_DATE,CONTRACT_CATEGORY,INSTALL_DATE1,Contract_length
0,Inactive - Unconfirmed,2004-05-07,0,10970,2004-02-17,No Contract,2004-05-07,0.00
1,Inactive - Unconfirmed,2004-04-16,0,10983,2004-03-15,No Contract,2004-04-16,0.00
2,Inactive - Unconfirmed,2004-04-23,0,10970,2004-04-23,No Contract,2004-04-23,0.00
3,Inactive - Unconfirmed,2004-08-16,0,10967,2004-03-24,No Contract,2004-08-16,0.00
4,Inactive - Unconfirmed,2004-07-23,0,10967,2004-03-24,No Contract,2004-07-23,0.00


### Aggregating

In [68]:
# copy columns to be aggregated
installbase2 = installbase[['INSTALL_DATE','INSTALL_DATE1','CONTRACT_FLAG','CUSTOMER_SITE_ID',
                            'INSTANCE_LAST_UPDATE_DATE','Contract_length', 'CONTRACT_CATEGORY']]

In [69]:
installbase2.head()

,INSTALL_DATE,INSTALL_DATE1,CONTRACT_FLAG,CUSTOMER_SITE_ID,INSTANCE_LAST_UPDATE_DATE,Contract_length,CONTRACT_CATEGORY
0,2004-05-07,2004-05-07,0,10970,2004-02-17,0.00,No Contract
1,2004-04-16,2004-04-16,0,10983,2004-03-15,0.00,No Contract
2,2004-04-23,2004-04-23,0,10970,2004-04-23,0.00,No Contract
3,2004-08-16,2004-08-16,0,10967,2004-03-24,0.00,No Contract
4,2004-07-23,2004-07-23,0,10967,2004-03-24,0.00,No Contract


In [70]:
# count number of install bases
installbase3 = installbase[['CUSTOMER_SITE_ID']]
installbase3 = installbase3.groupby('CUSTOMER_SITE_ID').size().reset_index()
installbase3 = installbase3.rename(columns = {0: 'Num_of_Install_Bases'})
installbase3.head()

,CUSTOMER_SITE_ID,Num_of_Install_Bases
0,24,5
1,36,43
2,85,1
3,86,10
4,90,6


In [71]:
# aggregate CONTRACT_FLAG, Contract_length, INSTANCE_LAST_UPDATE_DATE, INSTALL_DATE

installbase2['Contract_length'] = installbase2['Contract_length'].replace(0, np.nan)
installbase2 = installbase2.groupby('CUSTOMER_SITE_ID').agg({'CONTRACT_FLAG': 'sum',
                                                             'Contract_length': 'mean',     
}).reset_index()
# rename columns
installbase2 = installbase2.rename(columns = {"CONTRACT_FLAG": 'Total_Contracts'})

installbase2['Contract_length'] = installbase2['Contract_length'].replace(np.nan,0)
installbase2.head()

C:\Users\jayan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CUSTOMER_SITE_ID,Total_Contracts,Contract_length
0,24,0,0.00
1,36,11,625.00
2,85,0,0.00
3,86,4,"1,095.00"
4,90,6,"1,003.00"


In [72]:
# check all customer sites are unique
len(installbase2['CUSTOMER_SITE_ID'])/len(installbase2['CUSTOMER_SITE_ID'].unique())

1.0

In [73]:
# merge the above 2 tables
installbase4 = pd.merge(installbase2, installbase3, how = 'inner')
installbase4.head()

,CUSTOMER_SITE_ID,Total_Contracts,Contract_length,Num_of_Install_Bases
0,24,0,0.00,5
1,36,11,625.00,43
2,85,0,0.00,1
3,86,4,"1,095.00",10
4,90,6,"1,003.00",6


In [74]:
# aggregate STATUS
installbase['Num_of_Active_Install_Bases'] = installbase['STATUS'].map({'Inactive - Unconfirmed': 0,
                                                       'Inactive - Confirmed': 0,
                                                       'Active - Unconfirmed': 1,
                                                       'Active - Confirmed': 1})
installbase5 = installbase.groupby('CUSTOMER_SITE_ID')['Num_of_Active_Install_Bases'].sum().reset_index()
installbase5.head()

,CUSTOMER_SITE_ID,Num_of_Active_Install_Bases
0,24,5
1,36,39
2,85,0
3,86,6
4,90,6


In [75]:
# aggregate CONTRACT_CATEGORY
installbase6 = installbase.groupby('CUSTOMER_SITE_ID')['CONTRACT_CATEGORY'].apply(lambda x: x.mode()[0]).reset_index()
installbase6 = installbase6.rename(columns = {"CONTRACT_CATEGORY": 'Contract_Category'})
installbase6.head()

,CUSTOMER_SITE_ID,Contract_Category
0,24,No Contract
1,36,No Contract
2,85,No Contract
3,86,No Contract
4,90,FSMA


In [76]:
# merge the above tables
installbase = pd.merge(installbase5, installbase4, how = 'inner').merge(installbase6, how = 'inner')
installbase.head()

,CUSTOMER_SITE_ID,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Install_Bases,Contract_Category
0,24,5,0,0.00,5,No Contract
1,36,39,11,625.00,43,No Contract
2,85,0,0,0.00,1,No Contract
3,86,6,4,"1,095.00",10,No Contract
4,90,6,6,"1,003.00",6,FSMA


In [77]:
installbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20059 entries, 0 to 20058
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CUSTOMER_SITE_ID             20059 non-null  int64  
 1   Num_of_Active_Install_Bases  20059 non-null  int64  
 2   Total_Contracts              20059 non-null  int64  
 3   Contract_length              20059 non-null  float64
 4   Num_of_Install_Bases         20059 non-null  int64  
 5   Contract_Category            20059 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 1.1+ MB


In [78]:
# calculate a column called Number_of_Inactive_Sites
installbase['Num_of_Inactive_Install_Bases'] = installbase['Num_of_Install_Bases'] - installbase['Num_of_Active_Install_Bases']

In [79]:
installbase.head()

,CUSTOMER_SITE_ID,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Install_Bases,Contract_Category,Num_of_Inactive_Install_Bases
0,24,5,0,0.00,5,No Contract,0
1,36,39,11,625.00,43,No Contract,4
2,85,0,0,0.00,1,No Contract,1
3,86,6,4,"1,095.00",10,No Contract,4
4,90,6,6,"1,003.00",6,FSMA,0


In [80]:
# check all customer sites are unique
len(installbase['CUSTOMER_SITE_ID'])/len(installbase['CUSTOMER_SITE_ID'].unique())

1.0

# Exporting result tables

In [81]:
service.to_csv("service_aggregated.csv")
interactions.to_csv("interactions_aggregated.csv")
cases.to_csv("cases_aggregated.csv")
installbase.to_csv("installbase_aggregated.csv")